In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import os
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

c:\Users\ishan\AppData\Local\Programs\Python\Python310\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\ishan\AppData\Local\Programs\Python\Python310\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

In [2]:
df = pd.read_csv('MVP_Candidates.csv')
df.fillna(0)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,0,0,Nikola Jokić,C,5,26,DEN,74,74,10,...,105,117,144,55,230.9,286.9,1,1,0.875,2021-22
1,1,1,Joel Embiid,C,5,27,PHI,68,68,8,...,105,109,243,47,6.5,169.0,0,2,0.706,2021-22
2,2,2,Giannis Antetokounmpo,PF,4,27,MIL,67,67,7,...,93,112,248,49,125.1,211.1,0,3,0.595,2021-22
3,3,3,Devin Booker,SG,2,25,PHO,68,68,1,...,112,102,103,84,-4.1,32.6,0,4,0.216,2021-22
4,4,4,Luka Dončić,PG,1,22,DAL,65,65,5,...,96,101,140,102,-8.9,15.3,0,5,0.146,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,481,481,Bernard King,SF,3,34,WSB,64,64,20,...,103,99,100,30,-38.6,-26.0,0,16,0.007,1990-91
481,482,482,Kenny Smith,PG,1,25,HOU,78,78,8,...,110,112,106,164,115.5,148.1,0,17,0.005,1990-91
482,483,483,Hakeem Olajuwon,C,5,28,HOU,56,50,8,...,101,103,90,5,39.4,31.8,0,18,0.004,1990-91
483,484,484,Tim Hardaway,PG,1,24,GSW,82,82,11,...,105,102,77,198,63.4,44.2,0,19,0.001,1990-91


In [3]:
'''
1990-91 to 1998-99
1999-00 to 2011-12
2012-13 to 2021-22
'''
e1 = ('2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13')
e2 = ('2011-12', '2010-09', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00')
e3 = ('1998-99', '1997-98', '1996-97', '1995-96', '1994-95', '1993-94', '1992-93', '1991-92', '1990-91')


df1 = df[(df['Year'] == '1998-99') | (df['Year'] == '1997-98') | (df['Year'] == '1996-97') | (df['Year'] == '1995-96') | (df['Year'] == '1994-95') | (df['Year'] == '1993-94') | (df['Year'] == '1992-93') | (df['Year'] == '1991-92') | (df['Year'] == '1990-91')].reset_index(drop=True)


In [4]:
df1

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,326,326,Karl Malone,PF,4,35,UTA,49,49,2,...,108,113,183,1,44.0,132.8,1,1,0.701,1998-99
1,327,327,Alonzo Mourning,C,5,28,MIA,46,46,5,...,90,110,202,2,57.7,85.2,0,2,0.655,1998-99
2,328,328,Tim Duncan,PF,4,22,SAS,50,50,1,...,95,106,128,5,50.3,58.6,0,3,0.627,1998-99
3,329,329,Allen Iverson,SG,2,23,PHI,48,48,12,...,103,99,136,112,-55.2,-9.4,0,4,0.270,1998-99
4,330,330,Jason Kidd,PG,1,25,PHO,50,50,14,...,104,103,104,105,15.1,24.5,0,5,0.135,1998-99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,481,481,Bernard King,SF,3,34,WSB,64,64,20,...,103,99,100,30,-38.6,-26.0,0,16,0.007,1990-91
156,482,482,Kenny Smith,PG,1,25,HOU,78,78,8,...,110,112,106,164,115.5,148.1,0,17,0.005,1990-91
157,483,483,Hakeem Olajuwon,C,5,28,HOU,56,50,8,...,101,103,90,5,39.4,31.8,0,18,0.004,1990-91
158,484,484,Tim Hardaway,PG,1,24,GSW,82,82,11,...,105,102,77,198,63.4,44.2,0,19,0.001,1990-91


In [5]:
df1.drop(df1.columns[[0, 1]], axis = 1, inplace=True)

In [6]:
#df1['6.3P+'][108] = 0
''' WHY WAS THIS NaN I HAVE NO CLUE - FILLNA SHOULD HAVE MADE IT ZERO '''
''' ANYWAY PROBLEM SOLVED '''

' ANYWAY PROBLEM SOLVED '

In [7]:
df1.to_csv('OlderEra.csv', index=False)

In [8]:
df1

,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,Win %,1.MP,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,Karl Malone,PF,4,35,UTA,49,49,2,0.740000,1832,...,108,113,183,1,44.0,132.8,1,1,0.701,1998-99
1,Alonzo Mourning,C,5,28,MIA,46,46,5,0.660000,1753,...,90,110,202,2,57.7,85.2,0,2,0.655,1998-99
2,Tim Duncan,PF,4,22,SAS,50,50,1,0.740000,1963,...,95,106,128,5,50.3,58.6,0,3,0.627,1998-99
3,Allen Iverson,SG,2,23,PHI,48,48,12,0.560000,1990,...,103,99,136,112,-55.2,-9.4,0,4,0.270,1998-99
4,Jason Kidd,PG,1,25,PHO,50,50,14,0.540000,2060,...,104,103,104,105,15.1,24.5,0,5,0.135,1998-99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Bernard King,SF,3,34,WSB,64,64,20,0.365854,2401,...,103,99,100,30,-38.6,-26.0,0,16,0.007,1990-91
156,Kenny Smith,PG,1,25,HOU,78,78,8,0.634146,2699,...,110,112,106,164,115.5,148.1,0,17,0.005,1990-91
157,Hakeem Olajuwon,C,5,28,HOU,56,50,8,0.634146,2062,...,101,103,90,5,39.4,31.8,0,18,0.004,1990-91
158,Tim Hardaway,PG,1,24,GSW,82,82,11,0.536585,3215,...,105,102,77,198,63.4,44.2,0,19,0.001,1990-91


In [9]:
def calc_metrics(y_test, y_pred):
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    return rmse, r2

In [10]:
seasons = ['1998-99', '1997-98', '1996-97', '1995-96', '1994-95', '1993-94', '1992-93', '1991-92', '1990-91']

In [11]:
col_to_drop = []
for i in df1.columns:
    if i[0] == '3' or i[0] == '4':
        col_to_drop.append(i)

In [12]:
season_metrics = {season:[] for season in seasons}
all_results = pd.DataFrame(columns = ['Year', 'Player', 'MVP Share', 'MVP Rank', 'Predicted MVP Share'])

In [13]:
for season in seasons:
    
    testing_data = df1[(df1['Year'] == season)].copy()
    training_data = df1[(df1['Year'] != season)].copy()
    X_train = training_data.drop(['Player', 'Pos', 'Tm', 'MVP', 'MVP Rank', 'MVP Share', 'Year'] + col_to_drop, axis=1)
    Y_train = training_data['MVP Share']
    X_test = testing_data.drop(['Player', 'Pos', 'Tm', 'MVP', 'MVP Rank', 'MVP Share', 'Year'] + col_to_drop, axis=1)
    Y_test = testing_data['MVP Share']

    X_train.fillna(0)
    X_test.fillna(0)

    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    final_params = []

    param_grid = {'n_estimators': [5,10,20,30,40,50,100],
                    'learning_rate': [0.01,0.05,0.1,0.2,0.5]}
    ada_model = AdaBoostRegressor()
    grid = GridSearchCV(ada_model, param_grid)
    grid.fit(scaled_X_train, Y_train)
    model = AdaBoostRegressor(**grid.best_params_)
    final_params.append(grid.best_params_)

    model.fit(scaled_X_train, Y_train)

    Y_pred = model.predict(scaled_X_test)

    rmse, r2 = calc_metrics(Y_test, Y_pred)

    season_metrics[season] += [rmse, r2]

    initial_results = testing_data[['Year', 'Player', 'MVP Share', 'MVP Rank']]
    results = initial_results.copy()
    results['Predicted MVP Share'] = pd.Series(Y_pred).values
    results = results.sort_values('Predicted MVP Share', ascending=False).reset_index(drop=True)

    print(results)

    all_results = pd.concat([all_results, results])

       Year             Player  MVP Share  MVP Rank  Predicted MVP Share
0   1998-99   Shaquille O'Neal      0.075         6             0.097765
1   1998-99      Allen Iverson      0.270         4             0.035558
2   1998-99        Karl Malone      0.701         1             0.034344
3   1998-99     David Robinson      0.005        12             0.032110
4   1998-99      Kevin Garnett      0.008        10             0.032110
5   1998-99       Chris Webber      0.043         7             0.032110
6   1998-99         Grant Hill      0.033         8             0.029352
7   1998-99         Shawn Kemp      0.006        11             0.029352
8   1998-99    Alonzo Mourning      0.655         2             0.028937
9   1998-99         Tim Duncan      0.627         3             0.028937
10  1998-99        Gary Payton      0.030         9             0.027782
11  1998-99    Arvydas Sabonis      0.003        13             0.026766
12  1998-99  Darrell Armstrong      0.002        15

In [14]:
for i in season_metrics.keys():
    print(i)
    print("RMSE = ", season_metrics[i][0])
    print("R2 = ", season_metrics[i][1])
    print()

1998-99
RMSE =  0.24581767274734584
R2 =  -0.16215565795307207

1997-98
RMSE =  0.15723098504852576
R2 =  0.6318135764025614

1996-97
RMSE =  0.1074303553532296
R2 =  0.8182415284391894

1995-96
RMSE =  0.1075420171937424
R2 =  0.811305099060329

1994-95
RMSE =  0.15394062150756735
R2 =  0.6298000718058803

1993-94
RMSE =  0.17545936662415867
R2 =  0.5619805509199869

1992-93
RMSE =  0.22758105013236876
R2 =  0.3699563800215695

1991-92
RMSE =  0.14201890236950337
R2 =  0.6738474477430008

1990-91
RMSE =  0.10322408999627772
R2 =  0.8103730414478412



In [15]:
all_test = all_results['MVP Share']
all_pred = all_results['Predicted MVP Share']
total_rmse, total_r2 = calc_metrics(all_test, all_pred)
print("RMSE = ", total_rmse)
print("R2 = ", total_r2)

RMSE =  0.16443109095160313
R2 =  0.5766469732328586


In [16]:
print(all_results.to_string())

       Year             Player MVP Share MVP Rank Predicted MVP Share
0   1998-99   Shaquille O'Neal     0.075        6            0.097765
1   1998-99      Allen Iverson      0.27        4            0.035558
2   1998-99        Karl Malone     0.701        1            0.034344
3   1998-99     David Robinson     0.005       12             0.03211
4   1998-99      Kevin Garnett     0.008       10             0.03211
5   1998-99       Chris Webber     0.043        7             0.03211
6   1998-99         Grant Hill     0.033        8            0.029352
7   1998-99         Shawn Kemp     0.006       11            0.029352
8   1998-99    Alonzo Mourning     0.655        2            0.028937
9   1998-99         Tim Duncan     0.627        3            0.028937
10  1998-99        Gary Payton      0.03        9            0.027782
11  1998-99    Arvydas Sabonis     0.003       13            0.026766
12  1998-99  Darrell Armstrong     0.002       15            0.026766
13  1998-99       Ma